In [1]:
import sys, os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats

In [2]:
sys.argv[1] = 'trainingSet.csv'
sys.argv[2] = 'testSet.csv'

trainingDataFilename = sys.argv[1]
testDataFilename = sys.argv[2]
# modelIdx = int(sys.argv[3])
modelIdx = 1

In [3]:
DEPTH_LIMIT = 8
SAMPLE_LIMIT = 50
class TreeNode(object):
    def __init__(self, class_label, lineage):
        self.val = class_label
        self.lineage = lineage
        #self.attr = attr
        self.left = None
        self.right = None

    def is_leaf(self):
        return ((self.left is None) or (self.right is None))

    def expand(self, data, features,
               depth=DEPTH_LIMIT, downsampling_power=1):
#         if not len(data):
#             self.val = -1
#             return
#         class_col = data['decision']
#         print class_col
        class_label = int(len(data[data['decision'] == 1]) > len(data[data['decision'] == 0]))
        #print class_label
        available_features = [f for f in features if f not in self.lineage]
#         available_features = \
#             sample(eligible_features,
#                    int(round(len(eligible_features)**downsampling_power)))
        if len(self.lineage) < depth and len(data) > 50:
            ginis = {k: gini(data) for k in available_features}
            best_feature = max(ginis, key=ginis.get)
            #print best_feature, ginis[best_feature]
            left_data = data[data[best_feature] == 0]
            right_data = data[data[best_feature] == 1]
            
            self.val = best_feature
            self.left = TreeNode(class_label, self.lineage + [best_feature])
            self.left.expand(left_data, features, depth=depth,
                             downsampling_power=downsampling_power)
            self.right = TreeNode(class_label, self.lineage + [best_feature])
            self.right.expand(right_data, features, depth=depth,
                              downsampling_power=downsampling_power)
        else:
            #print 'leaf'
            self.val = class_label
            
def gini(data):
    gini = 0.0
    total = len(data)
    pos = 1.0 * data['decision'].value_counts()[0]/total
    neg = 1.0 * data['decision'].value_counts()[1]/total

    return 1 - pos ** 2 - neg ** 2, pos, neg

def get_gini_gain(data, attr):
    gain = gini(data) - pos * gini(data[data[attr] == 1]) - neg * gini(data[data[attr] == 1])
    return gain

In [4]:
def decisionTree(trainingSet, testSet, keys):
    print 'decisionTree'
    #ginis = {i: get_gini_gain(trainingSet, key) for key in keys}
    #best_feature = max(ginis, key=ginis.get)
    root = TreeNode(-1, [])
    root.expand(trainingSet, keys, DEPTH_LIMIT, 1)
    
    return root

def predict(node, data):
    if node.is_leaf():
        return node.val
    else:
        if data[node.val] == '0':
            predict(node.left, data)
        else:
            predict(node.right, data)

def get_accuracy_dt(root, trainingSet, testSet):
    count_train, total_train = 0, len(trainingSet)
    count_test, total_test = 0, len(testSet)
    
    train_labels = trainingSet['decision']
    test_labels = testSet['decision']
    
    trainingSet = trainingSet.drop('decision', axis=1)
    testSet = testSet.drop('decision', axis=1)
    
    predictions = []
    Y = np.array(train_labels)
    # Training accuracy
    for index, row in trainingSet.iterrows():
        predictions.append(predict(root, row)) 
        #print predict(root, row)
        if predictions[index] == int(Y[index]):
            count_train += 1

    training_accuracy = 1.0 * count_train/total_train
    print 'Training Accuracy DT:', '%.2f' % training_accuracy
    
    predictions = []
    Y = np.array(test_labels)

    # Training accuracy
    for index, row in testSet.iterrows():
        predictions.append(predict(root, row))
        #print predict(root, row)
        if predictions[index] == int(Y[index]):
            count_test += 1

    test_accuracy = 1.0 * count_test/total_test
    print 'Test Accuracy DT:', '%.2f' % test_accuracy

In [5]:
def bagging(trainingSet, testSet):
    print 'bagging'

In [6]:
def randomForests(trainingSet, testSet):
    print 'randomForests'

In [7]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

keys = trainingSet.keys()
keys = keys.drop('decision')

if modelIdx == 1:
    root = decisionTree(trainingSet, testSet, keys)
    print root.val
    get_accuracy_dt(root, trainingSet, testSet)
elif modelIdx == 2:
    bagging(trainingSet, testSet)
elif modelIdx == 3:
    randomForests(trainingSet, testSet)
else:
    print 'modelIdx error'

decisionTree
art
Training Accuracy DT: 0.00
Test Accuracy DT: 0.00
